# **Atividade Prática: Engenharia de Prompt para Geração Automática de Resumos**

**Objetivo:**  
Avaliar como diferentes formulações de prompts influenciam na qualidade dos resumos gerados por modelos abertos.  



## **1. Instalação das Dependências**

In [35]:
!pip install transformers sentencepiece rouge nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [36]:
import nltk
from rouge import Rouge
from transformers import pipeline
from nltk.translate.bleu_score import sentence_bleu

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/hericson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## **2. Preparação do Modelo Gratuito**

Utilizaremos o **modelo BART** da Meta (Facebook):

In [37]:
# Pipeline para sumarização automática
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


## **3. Texto a ser Resumido**

In [67]:
texto_original = """
Marte é o quarto planeta a partir do Sol, o segundo menor do Sistema Solar, atrás apenas de Mercúrio. Batizado em homenagem a divindade romana da guerra, muitas vezes é descrito como o "Planeta Vermelho", porque o óxido de ferro predominante em sua superfície lhe dá uma aparência avermelhada.Marte é um planeta rochoso com uma atmosfera fina, com características de superfície que lembram tanto as crateras de impacto da Lua quanto vulcões, vales, desertos e calotas polares da Terra.
"""

## **4. Definição de Prompts Alternativos**

Apesar do modelo open-source não usar prompt explícito igual GPT, podemos simular a influência do "prompt" adicionando instruções explícitas ao texto original.  



### Prompt 1 (Instrução direta no início):

In [68]:
prompt1 = "Resuma o texto de forma clara e objetiva: " + texto_original

### Prompt 2 (Prompt detalhado e contextualizado):



In [69]:
prompt2 = "Como especialista em resumos técnicos, produza um resumo curto e preciso contendo apenas as informações principais do seguinte texto: " + texto_original


## **5. Geração dos Resumos com BART**


In [70]:
def gerar_resumo(prompt):
    resumo = summarizer(prompt, max_length=130, min_length=30, do_sample=False)
    return resumo[0]['summary_text']

resumo1 = gerar_resumo(prompt1)
resumo2 = gerar_resumo(prompt2)

print("Resumo com Prompt 1:\n", resumo1)
print("\nResumo com Prompt 2:\n", resumo2)

Resumo com Prompt 1:
 Marte é o quarto planeta a partir do Sol, o segundo menor do Sistema Solar. Batizado em homenagem a divindade romana da guerra, muitas vezes é descrito como o "Planeta Vermelho"

Resumo com Prompt 2:
 Marte é o quarto planeta a partir do Sol, o segundo menor do Sistema Solar, atrás apenas de Mercúrio. Batizado em homenagem a divindade romana da guerra, muitas vezes é descrito como o "Planeta Vermelho"


## **6. Avaliação Quantitativa com Métricas ROUGE e BLEU**

Insira um resumo de referência para comparação (criado por você):

In [71]:
resumo_referencia = """
Marte é o quarto e segundo menor planeta do sistema solar. Foi batizado em homenagem a uma divindade romana da guerra. Sendo um planeta rochoso e com uma atmosfera fina, Marte apresenta características na sua superfície similares a da Lua e da Terra.
"""

**Cálculo das métricas:**

In [43]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/hericson/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [72]:
rouge = Rouge()

# Métricas ROUGE
scores_rouge1 = rouge.get_scores(resumo1, resumo_referencia)[0]
scores_rouge2 = rouge.get_scores(resumo2, resumo_referencia)[0]

print("Resultados ROUGE - Prompt 1:", scores_rouge1)
print("\nResultados ROUGE - Prompt 2:", scores_rouge2)

# Métrica BLEU
ref_tokens = [nltk.word_tokenize(resumo_referencia.lower())]

bleu1 = sentence_bleu(ref_tokens, nltk.word_tokenize(resumo1.lower()))
bleu2 = sentence_bleu(ref_tokens, nltk.word_tokenize(resumo2.lower()))

print("\nBLEU Prompt 1:", bleu1)
print("BLEU Prompt 2:", bleu2)

Resultados ROUGE - Prompt 1: {'rouge-1': {'r': 0.3888888888888889, 'p': 0.5384615384615384, 'f': 0.4516128983558793}, 'rouge-2': {'r': 0.18604651162790697, 'p': 0.26666666666666666, 'f': 0.21917807735034728}, 'rouge-l': {'r': 0.3611111111111111, 'p': 0.5, 'f': 0.41935483383975025}}

Resultados ROUGE - Prompt 2: {'rouge-1': {'r': 0.3888888888888889, 'p': 0.4666666666666667, 'f': 0.42424241928374656}, 'rouge-2': {'r': 0.18604651162790697, 'p': 0.23529411764705882, 'f': 0.20779220286051622}, 'rouge-l': {'r': 0.3611111111111111, 'p': 0.43333333333333335, 'f': 0.39393938898071623}}

BLEU Prompt 1: 0.208971500270483
BLEU Prompt 2: 0.188789354888571


### **Métricas que avaliam a semântica do texto e não somente N-grama, como as anteriores**

In [45]:
!pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [46]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

modelo_embedding = "sentence-transformers/all-MiniLM-L6-v2"
from sentence_transformers import SentenceTransformer
model_embedding = SentenceTransformer(modelo_embedding)

In [75]:
def similaridade_cosine(texto_gerado, texto_referencia):
    emb1 = model_embedding.encode(texto_gerado)
    emb2 = model_embedding.encode(texto_referencia)
    similaridade = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similaridade

In [76]:
similaridade_prompt1 = similaridade_cosine(resumo1, resumo_referencia)
similaridade_prompt2 = similaridade_cosine(resumo2, resumo_referencia)

print("Similaridade Semântica (Prompt 1):", similaridade_prompt1)
print("Similaridade Semântica (Prompt 2):", similaridade_prompt2)

Similaridade Semântica (Prompt 1): 0.90742743
Similaridade Semântica (Prompt 2): 0.89968586


## **7. Desafio Adicional: Avaliar com Métricas Complementares**

Exemplo simples: avaliar a concisão dos resumos (quantidade de palavras):

In [77]:
def contar_palavras(texto):
    return len(nltk.word_tokenize(texto))

print("\nPalavras no Resumo 1:", contar_palavras(resumo1))
print("Palavras no Resumo 2:", contar_palavras(resumo2))


Palavras no Resumo 1: 36
Palavras no Resumo 2: 41


## **8. Comparação de Outros Modelos Gratuitos (Opcional)**

Você pode repetir os passos acima usando outros modelos abertos:

Exemplos:

- **google/pegasus-xsum**
- **t5-base** (requer prompts específicos)

In [79]:
# Exemplo com outro modelo aberto:
summarizer_pegasus = pipeline("summarization", model="google/pegasus-xsum")
resumo_pegasus = summarizer(prompt1, max_length=150)[0]['summary_text']
print("Resumo com Pegasus:\n", resumo_pegasus)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Resumo com Pegasus:
 Marte é o quarto planeta a partir do Sol, o segundo menor do Sistema Solar. Batizado em homenagem a divindade romana da guerra, muitas vezes é descrito como o "Planeta Vermelho"


In [81]:
prompt1

'Resuma o texto de forma clara e objetiva: \nMarte é o quarto planeta a partir do Sol, o segundo menor do Sistema Solar, atrás apenas de Mercúrio. Batizado em homenagem a divindade romana da guerra, muitas vezes é descrito como o "Planeta Vermelho", porque o óxido de ferro predominante em sua superfície lhe dá uma aparência avermelhada.Marte é um planeta rochoso com uma atmosfera fina, com características de superfície que lembram tanto as crateras de impacto da Lua quanto vulcões, vales, desertos e calotas polares da Terra.\n'

In [80]:
resumo_pegasus = summarizer(prompt2, max_length=150)[0]['summary_text']
print("Resumo com Pegasus:\n", resumo_pegasus)

Resumo com Pegasus:
 Marte é o quarto planeta a partir do Sol, o segundo menor do Sistema Solar, atrás apenas de Mercúrio. Batizado em homenagem a divindade romana da guerra, muitas vezes é descrito como o "Planeta Vermelho"


Depois compare métricas (ROUGE e BLEU) com as obtidas anteriormente.

In [82]:
similaridade_prompt1 = similaridade_cosine(resumo_pegasus, resumo_referencia)

print("Similaridade Semântica (pegasus) (Prompt 1):", similaridade_prompt1)

Similaridade Semântica (pegasus) (Prompt 1): 0.89968586


---

## **Conclusão da atividade**

Ao final, analise e responda:

- Qual prompt gerou o melhor resultado?
- Houve influência significativa da estrutura do prompt no resultado?
- Como os modelos avaliados se comparam?




# **Atividade Prática adicional: Engenharia de Prompt para geração automática de Relatórios Técnicos**

**Objetivo:**  
Avaliar como diferentes prompts influenciam a qualidade de relatórios técnicos gerados automaticamente por modelos gratuitos (como o BART), utilizando métricas quantitativas (ROUGE, BLEU) para comparação objetiva.  

---

## **1. Instalação das dependências**

In [51]:
!pip install transformers rouge nltk

import nltk
from transformers import pipeline
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu

nltk.download('punkt')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[nltk_data] Downloading package punkt to /home/hericson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## **2. Carregar Modelo Gratuito (BART)**




In [ ]:
generator = pipeline("text2text-generation", model="facebook/bart-large-cnn")

Device set to use cpu


## **3. Texto-base para o relatório técnico**

Insira aqui o texto base (fictício ou real) sobre o qual será gerado o relatório:

In [104]:
texto_original = """
A órbita de Marte ao redor do Sol é extremamente elíptica. E porque a distância entre o Sol e Marte varia, a temperatura varia desde -125 graus Celsius no inverno Marciano até 22 graus Celsius no verão Marciano. A atmosfera Marciana é composta de mais de 95% de dióxido de carbono. Ventos solares dispersam a fina e fraca atmosfera porque Marte tem campos gravitacional e magnético muito fracos. Nos pólos Marcianos encontram-se capas de gelo polar que diminuem de tamanho durante a primavera e verão Marcianos. A partir de dados coletados pelas sondas Viking 1 e 2, sabemos que a superfície Marciana é coberta por muitas rochas e solo rico em uma argila carregada de ferro. A presença de ferro explica a aparência vermelha do planeta. Marte contém terras altas que ocorrem no hemisfério Sul e são compostas por material mais velho, endurecido, e com mais crateras. Marte também contém terras baixas que são encontradas no hemisfério Norte. 
"""

## **4. Desenvolvimento dos Prompts para Geração**

**Prompt 1 (Direto e simples)**

In [105]:
prompt1 = f"""
Gere um relatório técnico curto e objetivo baseado no texto abaixo:

{texto_original}

Relatório técnico:
"""


**Prompt 2 (Contextualizado e detalhado)**

In [106]:
prompt2 = f"""
Você é um engenheiro especialista responsável pela elaboração de relatórios técnicos precisos e detalhados, a partir do texto abaixo, escreva um breve relatório técnico destacando claramente objetivo, metodologia, principais resultados e conclusões:

{texto_original}

Relatório técnico:
"""

In [107]:
prompt2

'\nVocê é um engenheiro especialista responsável pela elaboração de relatórios técnicos precisos e detalhados, a partir do texto abaixo, escreva um breve relatório técnico destacando claramente objetivo, metodologia, principais resultados e conclusões:\n\n\nA órbita de Marte ao redor do Sol é extremamente elíptica. E porque a distância entre o Sol e Marte varia, a temperatura varia desde -125 graus Celsius no inverno Marciano até 22 graus Celsius no verão Marciano. A atmosfera Marciana é composta de mais de 95% de dióxido de carbono. Ventos solares dispersam a fina e fraca atmosfera porque Marte tem campos gravitacional e magnético muito fracos. Nos pólos Marcianos encontram-se capas de gelo polar que diminuem de tamanho durante a primavera e verão Marcianos. A partir de dados coletados pelas sondas Viking 1 e 2, sabemos que a superfície Marciana é coberta por muitas rochas e solo rico em uma argila carregada de ferro. A presença de ferro explica a aparência vermelha do planeta. Marte 

## **4. Gerar Relatórios Técnicos com os prompts**

In [117]:
def gerar_relatorio(prompt):
    resultado = generator(prompt, max_length=250, do_sample=False)
    return resultado[0]['generated_text']

relatorio1 = gerar_relatorio(prompt1)
relatorio2 = gerar_relatorio(prompt2)

print("Relatório (Prompt 1):\n", relatorio1)
print("\nRelatório (Prompt 2):\n", relatorio2)


Relatório (Prompt 1):
 A atmosfera Marciana é composta de mais de 95% de dióxido de carbono. Ventos solares dispersam a fina e fraca atmosFERa porque Marte tem campos gravitacional e magnético fracos. Nos pólos Marcianos encontram-se capas de gelo polar that diminuem de tamanho durante a primavera e verão.

Relatório (Prompt 2):
 Você é um engenheiro especialista responsável pela elaboração de relatórios técnicos. Escreva um breve relatório técnico destacando claramente objetivo, metodologia, principais resultados e conclusões: A órbita de Marte ao redor do Sol é extremamente elíptica.



## **4. Avaliação quantitativa dos resultados (ROUGE e BLEU)**

### **Defina primeiro um relatório técnico de referência (feito manualmente):**

In [109]:
relatorio_referencia = """
Este relatório técnico apresenta uma análise das características físicas e atmosféricas do planeta Marte, com base em dados coletados por diversas missões espaciais, incluindo as sondas Viking 1 e 2. O objetivo é fornecer uma visão geral das condições que moldam o ambiente marciano."""


**Métricas:**


In [110]:
from rouge import Rouge
from nltk.tokenize import word_tokenize

rouge = Rouge()

# Avaliação com ROUGE
rouge_relatorio1 = rouge.get_scores(relatorio1, relatorio_referencia)[0]
rouge_relatorio2 = rouge.get_scores(relatorio2, relatorio_referencia)[0]

print("ROUGE - Prompt 1:", rouge_relatorio1)
print("ROUGE - Prompt 2:", rouge_relatorio2)

# Avaliação com BLEU
bleu_prompt1 = sentence_bleu([word_tokenize(relatorio1.lower())], word_tokenize(relatorio1.lower()))
bleu_prompt2 = sentence_bleu([word_tokenize(relatorio2.lower())], word_tokenize(relatorio2.lower()))

print("\nBLEU Prompt 1:", bleu_prompt1)
print("BLEU Prompt 2:", bleu_prompt2)

ROUGE - Prompt 1: {'rouge-1': {'r': 0.04878048780487805, 'p': 0.05263157894736842, 'f': 0.050632906399615936}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.04878048780487805, 'p': 0.05263157894736842, 'f': 0.050632906399615936}}
ROUGE - Prompt 2: {'rouge-1': {'r': 0.12195121951219512, 'p': 0.15625, 'f': 0.13698629644586244}, 'rouge-2': {'r': 0.023255813953488372, 'p': 0.029411764705882353, 'f': 0.025974021042335226}, 'rouge-l': {'r': 0.12195121951219512, 'p': 0.15625, 'f': 0.13698629644586244}}

BLEU Prompt 1: 1.0
BLEU Prompt 2: 1.0


#### **Metricas semânticas**

In [111]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

modelo_embedding = "sentence-transformers/all-MiniLM-L6-v2"
from sentence_transformers import SentenceTransformer
model_embedding = SentenceTransformer(modelo_embedding)

In [ ]:
def similaridade_cosine(texto_gerado, texto_referencia):
    emb1 = model_embedding.encode(texto_gerado)
    emb2 = model_embedding.encode(texto_referencia)
    similaridade = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similaridade

In [113]:
similaridade_prompt1 = similaridade_cosine(relatorio1, relatorio_referencia)
similaridade_prompt2 = similaridade_cosine(relatorio2, relatorio_referencia)

print("Similaridade Semântica (Prompt 1):", similaridade_prompt1)
print("Similaridade Semântica (Prompt 2):", similaridade_prompt2)


Similaridade Semântica (Prompt 1): 0.4257745
Similaridade Semântica (Prompt 2): 0.49457496


## **4. Desafio adicional (opcional): outras métricas**

Você pode utilizar métricas adicionais:

- Avaliação da concisão (contagem de palavras)
- Avaliação qualitativa subjetiva (clareza técnica)

Exemplo: contagem simples de palavras:

In [116]:
def contar_palavras(texto):
    return len(word_tokenize(texto))

print("\nNúmero de palavras Relatório 1:", contar_palavras(relatorio1))
print("Número de palavras Relatório 2:", contar_palavras(relatorio2))



Número de palavras Relatório 1: 50
Número de palavras Relatório 2: 40


## **5. Comparação de diferentes modelos (Opcional)**

Você pode também comparar com modelos como T5 ou Pegasus para geração:

In [118]:
generator_t5 = pipeline("text2text-generation", model="google/flan-t5-base")

# Exemplo com prompt1 usando T5
relatorio_t5 = generator(prompt1, max_length=250, do_sample=False)[0]['generated_text']
print("\nRelatório Técnico (T5):\n", relatorio_t5)


Device set to use cpu



Relatório Técnico (T5):
 A atmosfera Marciana é composta de mais de 95% de dióxido de carbono. Ventos solares dispersam a fina e fraca atmosFERa porque Marte tem campos gravitacional e magnético fracos. Nos pólos Marcianos encontram-se capas de gelo polar that diminuem de tamanho durante a primavera e verão.


In [119]:
similaridade_prompt1 = similaridade_cosine(relatorio1, relatorio_t5)

print("Similaridade Semântica (Prompt 1):", similaridade_prompt1)


Similaridade Semântica (Prompt 1): 1.0000001


Faça nova avaliação (ROUGE e BLEU) para comparação quantitativa entre modelos.


## **Conclusão da atividade**

Analise e responda:

- Qual prompt apresentou melhor desempenho nas métricas usadas?
- As métricas quantitativas foram suficientes para avaliar a qualidade técnica dos relatórios gerados?
- Que tipo de prompt se mostrou mais eficaz na tarefa técnica?

